In [2]:
import redis
import os

user_connection = redis.Redis(host='localhost', port=6379, password=os.environ['REDIS_HOST_PASSWORD'], decode_responses=True)

user_connection.ping()


True

In [3]:
#儲存一個key
user_connection.set("full_name", "john doe")

True

In [4]:
#檢查是否有key存在
user_connection.exists("full_name")

1

In [5]:
#取出key的值
user_connection.get("full_name")

'john doe'

In [6]:
#覆蓋原來key的值
user_connection.set("full_name","徐國堂")
user_connection.get("full_name")

'徐國堂'

In [7]:
#設定有限定時間的key
#目前時間後100秒
user_connection.setex("important_key", 100, "important_value")

True

In [8]:
#剩82秒
user_connection.ttl("important_key")

100

In [9]:
#直接儲存dict
dict_data = {
    "employee_name": "Adam Adams",
    "employee_age": 30,
    "position": "Software Engineer",
}

user_connection.mset(dict_data)

True

In [10]:
#直接取出所有資料
user_connection.mget("employee_name", "employee_age", "position", "non_existing")


['Adam Adams', '30', 'Software Engineer', None]

In [11]:

import redis
from redis.commands.json.path import Path
import os

client = redis.Redis(host='localhost', port=6379, password=os.environ['REDIS_HOST_PASSWORD'], decode_responses=True)

jane = {
     'name': "Jane",
     'Age': 33,
     'Location': "Chawton"
   }

client.json().set('person:1', '$', jane)

result = client.json().get('person:1')
print(result)

{'name': 'Jane', 'Age': 33, 'Location': 'Chawton'}


In [13]:
import redis
from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import NumericFilter, Query


r = redis.Redis(host='localhost', port=6379, password=os.environ['REDIS_HOST_PASSWORD'], decode_responses=True)

user1 = {
    "user":{
        "name": "Paul John",
        "email": "paul.john@example.com",
        "age": 42,
        "city": "London"
    }
}
user2 = {
    "user":{
        "name": "Eden Zamir",
        "email": "eden.zamir@example.com",
        "age": 29,
        "city": "Tel Aviv"
    }
}
user3 = {
    "user":{
        "name": "Paul Zamir",
        "email": "paul.zamir@example.com",
        "age": 35,
        "city": "Tel Aviv"
    }
}
r.json().set("user:1", Path.root_path(), user1)
r.json().set("user:2", Path.root_path(), user2)
r.json().set("user:3", Path.root_path(), user3)

schema = (TextField("$.user.name", as_name="name"),TagField("$.user.city", as_name="city"), NumericField("$.user.age", as_name="age"))
r.ft().create_index(schema, definition=IndexDefinition(prefix=["user:"], index_type=IndexType.JSON))

'OK'